# Testing import class for Vertex AI Vector Search

In [1]:
import os

root_path = '..'
os.chdir(root_path)
os.getcwd()

'/home/jupyter/vector-io'

In [2]:
!pwd

/home/jupyter/vector-io


In [3]:
# naming convention for all cloud resources
VERSION        = "pubv3"                     # TODO
PREFIX         = f'vvs-vectorio-{VERSION}'   # TODO

print(f"PREFIX = {PREFIX}")

PREFIX = vvs-vectorio-pubv3


## Load notebook config

In [4]:
# staging GCS
GCP_PROJECTS             = !gcloud config get-value project
PROJECT_ID               = GCP_PROJECTS[0]

# GCS bucket and paths
BUCKET_NAME              = f'{PREFIX}-{PROJECT_ID}'
BUCKET_URI               = f'gs://{BUCKET_NAME}'

config = !gsutil cat {BUCKET_URI}/config/notebook_env.py
print(config.n)
exec(config.n)


PREFIX                   = "vvs-vectorio-pubv3"
VERSION                  = "pubv3"

PROJECT_ID               = "hybrid-vertex"
PROJECT_NUM              = "934903580331"

REGION                   = "us-central1"
BQ_REGION                = "US"

VERTEX_SA                = "934903580331-compute@developer.gserviceaccount.com"

VPC_NETWORK_NAME         = ""
VPC_NETWORK_FULL         = "None"

USE_PUBLIC_ENDPOINTS     = "True"

BUCKET_NAME              = "vvs-vectorio-pubv3-hybrid-vertex"
BUCKET_URI               = "gs://vvs-vectorio-pubv3-hybrid-vertex"

REMOTE_GCS_FOLDER        = "gs://vvs-vectorio-pubv3-hybrid-vertex/vvs-vectorio-pubv3/embedding_indexes/tmpig2laxe1/"

SO_PARQUET_GCS_DIR       = "gs://vvs-vectorio-pubv3-hybrid-vertex/emb_vector_parquet/so_0_1000_200/tmpnb32rkne/"

LOCAL_TEST_DIR           = "data/stack_overflow_parquet"
LOCAL_TEST_DATA_DIR      = "data/stack_overflow_parquet/files"

DIMENSIONS               = "768"

INDEX_DISPLAY_NAME       = "soverflow_vvs_vectorio_pubv3"

In [5]:
# LOCAL_PARQUEST_FILE_LIST = LOCAL_PARQUEST_FILE_LIST.split("|")
# LOCAL_PARQUEST_FILE_LIST

In [6]:
# tmp
LOCAL_PARQUEST_FILE_LIST = [
    'data/stack_overflow_parquet_v2/files/so_tmp3328v78o_1500.parquet',
    'data/stack_overflow_parquet_v2/files/so_tmp3328v78o_1501.parquet',
    'data/stack_overflow_parquet_v2/files/so_tmp3328v78o_1503.parquet',
    'data/stack_overflow_parquet_v2/files/so_tmp3328v78o_1502.parquet',
    'data/stack_overflow_parquet_v2/files/so_tmp3328v78o_1504.parquet'
]
LOCAL_PARQUEST_FILE_LIST

['data/stack_overflow_parquet_v2/files/so_tmp3328v78o_1500.parquet',
 'data/stack_overflow_parquet_v2/files/so_tmp3328v78o_1501.parquet',
 'data/stack_overflow_parquet_v2/files/so_tmp3328v78o_1503.parquet',
 'data/stack_overflow_parquet_v2/files/so_tmp3328v78o_1502.parquet',
 'data/stack_overflow_parquet_v2/files/so_tmp3328v78o_1504.parquet']

In [7]:
LOCAL_TEST_DIR = 'data/stack_overflow_parquet_v2'
LOCAL_TEST_DATA_DIR = 'data/stack_overflow_parquet_v2/files'

## Imports

In [8]:
import pandas as pd
import numpy as np
import itertools
import time 
import json
import uuid

from pprint import pprint

from google.cloud import aiplatform as aip
from google.cloud import storage
from google.cloud import bigquery

# logging
import logging
logging.disable(logging.WARNING)

#python warning 
import warnings
warnings.filterwarnings("ignore")

print(f'BigQuery SDK version      : {bigquery.__version__}')
print(f'Vertex AI SDK version     : {aip.__version__}')
print(f'Cloud Storage SDK version : {storage.__version__}')

BigQuery SDK version      : 3.15.0
Vertex AI SDK version     : 1.39.0
Cloud Storage SDK version : 2.14.0


### init Google Cloud SDK clients

In [9]:
# cloud storage client
storage_client = storage.Client(project=PROJECT_ID)

# Vertex client
aip.init(project=PROJECT_ID, location=REGION)

### Vertex AI Vector Search Import class

In [10]:
import sys
import os

# sys.path.append("..")
from src.import_vdf.vertexai_vector_search_import import ImportVertexAIVectorSearch
from src.names import DBNames

## Get test data

In [11]:
!tree notebooks/data

notebooks/data
├── 1500_2000_100
│   ├── VDF_META.json
│   └── tmpqxi_pbxv
│       ├── tmpmee85mkz_1501.parquet
│       ├── tmpmee85mkz_1502.parquet
│       ├── tmpmee85mkz_1503.parquet
│       └── tmpmee85mkz_1504.parquet
├── old_jt
│   └── so_1500_2000_100.parquet
├── stack_overflow_parquet
│   ├── VDF_META.json
│   └── files
│       ├── so_tmpig2laxe1_0.parquet
│       ├── so_tmpig2laxe1_1.parquet
│       ├── so_tmpig2laxe1_2.parquet
│       ├── so_tmpig2laxe1_3.parquet
│       └── so_tmpig2laxe1_4.parquet
└── stack_overflow_parquet_v2
    ├── VDF_META.json
    └── files
        ├── so_tmp3328v78o_1500.parquet
        ├── so_tmp3328v78o_1501.parquet
        ├── so_tmp3328v78o_1502.parquet
        ├── so_tmp3328v78o_1503.parquet
        └── so_tmp3328v78o_1504.parquet

7 directories, 18 files


In [12]:
# LOCAL_TEST_DIR            # 'data/stack_overflow_parquet'
# LOCAL_TEST_DATA_DIR       # 'data/stack_overflow_parquet/files'

# local vdf json file
TEST_VDF_META = f"notebooks/{LOCAL_TEST_DIR}/VDF_META.json"

VDF_VERSION = "v1"
TIMESTAMP_vdf = time.strftime("%Y%m%d-%H%M%S")

print(f"TEST_VDF_META : {TEST_VDF_META}")
print(f"DIMENSIONS    : {DIMENSIONS}")
print(f"VDF_VERSION   : {VDF_VERSION}")
print(f"TIMESTAMP_vdf : {TIMESTAMP_vdf}\n")

FILE_STRUCTURE_V1 = []
FILE_STRUCTURE_V1.append(f"{TEST_VDF_META}")
for element in LOCAL_PARQUEST_FILE_LIST:
    FILE_STRUCTURE_V1.append(f"notebooks/{element}")

FILE_STRUCTURE_V1

TEST_VDF_META : notebooks/data/stack_overflow_parquet_v2/VDF_META.json
DIMENSIONS    : 768
VDF_VERSION   : v1
TIMESTAMP_vdf : 20240130-184949



['notebooks/data/stack_overflow_parquet_v2/VDF_META.json',
 'notebooks/data/stack_overflow_parquet_v2/files/so_tmp3328v78o_1500.parquet',
 'notebooks/data/stack_overflow_parquet_v2/files/so_tmp3328v78o_1501.parquet',
 'notebooks/data/stack_overflow_parquet_v2/files/so_tmp3328v78o_1503.parquet',
 'notebooks/data/stack_overflow_parquet_v2/files/so_tmp3328v78o_1502.parquet',
 'notebooks/data/stack_overflow_parquet_v2/files/so_tmp3328v78o_1504.parquet']

In [13]:
# validate parquet files
df_from_pq = pd.read_parquet(FILE_STRUCTURE_V1[1])

print(df_from_pq.shape)
df_from_pq.head(3)

(100, 5)


,id,embedding,tag,score,crowding_tag
0,1869123,"[0.004282408859580755, -0.01166266854852438, 0...",wpf,2,frameworkelement
1,1948390,"[0.040322981774806976, 0.007929936982691288, 0...",windows-7,2,restriction
2,1953791,"[0.004021648317575455, -0.019918881356716156, ...",sql,2,full-text-search


### Create VDF Metadata json

In [14]:
target_vs_index = aip.MatchingEngineIndex(INDEX_RESOURCE_NAME)
target_vs_index.to_dict()

{'name': 'projects/934903580331/locations/us-central1/indexes/1081325705452584960',
 'displayName': 'soverflow_vvs_vectorio_pubv3',
 'description': 'sample index for vectorio demo',
 'metadataSchemaUri': 'gs://google-cloud-aiplatform/schema/matchingengine/metadata/nearest_neighbor_search_1.0.0.yaml',
 'metadata': {'config': {'dimensions': 768.0,
   'approximateNeighborsCount': 150.0,
   'distanceMeasureType': 'DOT_PRODUCT_DISTANCE',
   'algorithmConfig': {'treeAhConfig': {'leafNodeEmbeddingCount': '500',
     'leafNodesToSearchPercent': 80.0}},
   'shardSize': 'SHARD_SIZE_MEDIUM'}},
 'deployedIndexes': [{'indexEndpoint': 'projects/934903580331/locations/us-central1/indexEndpoints/5739455095037231104',
   'deployedIndexId': 'soverflow_vvs_vectorio_pubv3_20240130131739'}],
 'etag': 'AMEw9yPF_gXj1ARAX6KLL8qg3_KloDJxKCnjr3Gi65A7038lkqZWOz2jV-WEcFrTCc0=',
 'createTime': '2024-01-30T13:26:08.725251Z',
 'updateTime': '2024-01-30T13:26:17.176312Z',
 'indexStats': {'vectorsCount': '1', 'shardsC

In [15]:
my_vdf = {
    "author": "jordantotten",
    "exported_from": "vertex",
    "file_structure": FILE_STRUCTURE_V1,
    "version": VDF_VERSION,
    "exported_at": TIMESTAMP_vdf,
    "indexes": {
            "test_so_index": [
                {
                    "data_path": f"notebooks/{LOCAL_TEST_DATA_DIR}",
                    "dimensions": DIMENSIONS,
                    "exported_vector_count": 1000,
                    "metric": "Dot",
                    "model_name": "textembedding-gecko@001",
                    "namespace": "so_questions",
                    "total_vector_count": 1000,
                    "vector_columns": ["embedding"]
                }
            ]
    }
}
pprint(my_vdf)

{'author': 'jordantotten',
 'exported_at': '20240130-184949',
 'exported_from': 'vertex',
 'file_structure': ['notebooks/data/stack_overflow_parquet_v2/VDF_META.json',
                    'notebooks/data/stack_overflow_parquet_v2/files/so_tmp3328v78o_1500.parquet',
                    'notebooks/data/stack_overflow_parquet_v2/files/so_tmp3328v78o_1501.parquet',
                    'notebooks/data/stack_overflow_parquet_v2/files/so_tmp3328v78o_1503.parquet',
                    'notebooks/data/stack_overflow_parquet_v2/files/so_tmp3328v78o_1502.parquet',
                    'notebooks/data/stack_overflow_parquet_v2/files/so_tmp3328v78o_1504.parquet'],
 'indexes': {'test_so_index': [{'data_path': 'notebooks/data/stack_overflow_parquet_v2/files',
                                'dimensions': '768',
                                'exported_vector_count': 1000,
                                'metric': 'Dot',
                                'model_name': 'textembedding-gecko@001',
        

*Write local VDF metadata json file*

In [16]:
with open(f"{TEST_VDF_META}", 'w') as fp:
    json.dump(my_vdf, fp)

*Validate json file*

In [17]:
with open(f"{TEST_VDF_META}") as f:
    d = json.load(f)
    pprint(d)

{'author': 'jordantotten',
 'exported_at': '20240130-184949',
 'exported_from': 'vertex',
 'file_structure': ['notebooks/data/stack_overflow_parquet_v2/VDF_META.json',
                    'notebooks/data/stack_overflow_parquet_v2/files/so_tmp3328v78o_1500.parquet',
                    'notebooks/data/stack_overflow_parquet_v2/files/so_tmp3328v78o_1501.parquet',
                    'notebooks/data/stack_overflow_parquet_v2/files/so_tmp3328v78o_1503.parquet',
                    'notebooks/data/stack_overflow_parquet_v2/files/so_tmp3328v78o_1502.parquet',
                    'notebooks/data/stack_overflow_parquet_v2/files/so_tmp3328v78o_1504.parquet'],
 'indexes': {'test_so_index': [{'data_path': 'notebooks/data/stack_overflow_parquet_v2/files',
                                'dimensions': '768',
                                'exported_vector_count': 1000,
                                'metric': 'Dot',
                                'model_name': 'textembedding-gecko@001',
        

# Initialize import class

> Pass config for target index to import vectors t

In [18]:
my_import_args = {
    "project_id": PROJECT_ID,
    "location": REGION,
    "project_num" : PROJECT_NUM,
    "target_index_id": MY_INDEX_ID,
    "batch_size": 100,
    "library_version": VDF_VERSION,
    "dir": f"notebooks/{LOCAL_TEST_DIR}",
    "filter_restricts": [
        {
            "namespace": "tag",     # vertex VS namespace
            "allow_list": ["tag"],  # col name
        },
    ],
    "numeric_restricts" : [
        {
            "namespace": "score", 
            "data_type": "value_int"
        }
    ],
    "crowding_tag" : "crowding_tag"
}
pprint(my_import_args)

{'batch_size': 100,
 'crowding_tag': 'crowding_tag',
 'dir': 'notebooks/data/stack_overflow_parquet_v2',
 'filter_restricts': [{'allow_list': ['tag'], 'namespace': 'tag'}],
 'library_version': 'v1',
 'location': 'us-central1',
 'numeric_restricts': [{'data_type': 'value_int', 'namespace': 'score'}],
 'project_id': 'hybrid-vertex',
 'project_num': '934903580331',
 'target_index_id': '1081325705452584960'}


In [19]:
import_vvs = ImportVertexAIVectorSearch(
    args=my_import_args 
)

import_vvs

list_of_ns_restrict_entries : [{'namespace': 'tag', 'allow_list': ['tag']}]
list_of_numeric_entries : [{'namespace': 'score', 'data_type': 'value_int'}]
Importing to index : soverflow_vvs_vectorio_pubv3
Full resource name : projects/934903580331/locations/us-central1/indexes/1081325705452584960
Target index config:
{
    "dimensions": 768.0,
    "approximateNeighborsCount": 150.0,
    "distanceMeasureType": "DOT_PRODUCT_DISTANCE",
    "algorithmConfig": {
        "treeAhConfig": {
            "leafNodeEmbeddingCount": "500",
            "leafNodesToSearchPercent": 80.0
        }
    },
    "shardSize": "SHARD_SIZE_MEDIUM"
}


In [20]:
import_vvs.vdf_meta

{'author': 'jordantotten',
 'exported_from': 'vertex',
 'file_structure': ['notebooks/data/stack_overflow_parquet_v2/VDF_META.json',
  'notebooks/data/stack_overflow_parquet_v2/files/so_tmp3328v78o_1500.parquet',
  'notebooks/data/stack_overflow_parquet_v2/files/so_tmp3328v78o_1501.parquet',
  'notebooks/data/stack_overflow_parquet_v2/files/so_tmp3328v78o_1503.parquet',
  'notebooks/data/stack_overflow_parquet_v2/files/so_tmp3328v78o_1502.parquet',
  'notebooks/data/stack_overflow_parquet_v2/files/so_tmp3328v78o_1504.parquet'],
 'version': 'v1',
 'exported_at': '20240130-184949',
 'indexes': {'test_so_index': [{'data_path': 'notebooks/data/stack_overflow_parquet_v2/files',
    'dimensions': '768',
    'exported_vector_count': 1000,
    'metric': 'Dot',
    'model_name': 'textembedding-gecko@001',
    'namespace': 'so_questions',
    'total_vector_count': 1000,
    'vector_columns': ['embedding']}]}}

In [21]:
import_vvs.upsert_data()

Importing data from: test_so_index
index_meta: [{'data_path': 'notebooks/data/stack_overflow_parquet_v2/files', 'dimensions': '768', 'exported_vector_count': 1000, 'metric': 'Dot', 'model_name': 'textembedding-gecko@001', 'namespace': 'so_questions', 'total_vector_count': 1000, 'vector_columns': ['embedding']}]
data_path: notebooks/data/stack_overflow_parquet_v2/files
vector_column_name    : embedding
vector_metadata_names : ['embedding']


Inserting data: 100%|██████████| 5/5 [00:02<00:00,  1.94it/s]

Index import complete
Updated soverflow_vvs_vectorio_pubv3 with 500 vectors


In [22]:
target_vs_index.to_dict()

{'name': 'projects/934903580331/locations/us-central1/indexes/1081325705452584960',
 'displayName': 'soverflow_vvs_vectorio_pubv3',
 'description': 'sample index for vectorio demo',
 'metadataSchemaUri': 'gs://google-cloud-aiplatform/schema/matchingengine/metadata/nearest_neighbor_search_1.0.0.yaml',
 'metadata': {'config': {'dimensions': 768.0,
   'approximateNeighborsCount': 150.0,
   'distanceMeasureType': 'DOT_PRODUCT_DISTANCE',
   'algorithmConfig': {'treeAhConfig': {'leafNodeEmbeddingCount': '500',
     'leafNodesToSearchPercent': 80.0}},
   'shardSize': 'SHARD_SIZE_MEDIUM'}},
 'deployedIndexes': [{'indexEndpoint': 'projects/934903580331/locations/us-central1/indexEndpoints/5739455095037231104',
   'deployedIndexId': 'soverflow_vvs_vectorio_pubv3_20240130131739'}],
 'etag': 'AMEw9yPF_gXj1ARAX6KLL8qg3_KloDJxKCnjr3Gi65A7038lkqZWOz2jV-WEcFrTCc0=',
 'createTime': '2024-01-30T13:26:08.725251Z',
 'updateTime': '2024-01-30T13:26:17.176312Z',
 'indexStats': {'vectorsCount': '1', 'shardsC

In [21]:
# target_vs_index.

## Remove data

> TODO

In [ ]:
# parquet_files = sorted(
#     [file for file in os.listdir(data_path) if file.endswith(".parquet")]
# )

# Notes

### Format vectors

```
        # Append to file
        embeddings_formatted = [
            json.dumps(
                {
                    "id": str(id),
                    "embedding": [
                        str(value) for value in embedding
                    ],
                    "restricts": [
                        {"namespace": "tags", "allow": [str(tag_r)]}
                    ],
                    "numeric_restricts": [
                        {"namespace": "score", "value_int": int(score)}
                    ],
                    "crowding_tag": str(tag_crowd)
                }
            )
            + "\n"
            # for id, embedding in zip(id_chunk[is_successful], question_chunk_embeddings)
            for id, embedding, tag_r, score, tag_crowd in zip(
                id_chunk[is_successful], 
                question_chunk_embeddings, 
                tags_restrict, 
                scores_chunk, 
                tags_crowd
            )
        ]
        f.writelines(embeddings_formatted)
```

**Finished**